# Click clustering
In this notebook we plan to cluster the students in each course by their click activity. This is to see if we can create clusters that might belong more strongly to pass or fail. 

In [11]:
import pandas as pd
import numpy as np
import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression

from collections import Counter

import math

In [12]:
import matplotlib.pyplot as plt

# Load data into panda-frames and inspect

In [13]:
filenames = glob.glob("../data/*")

In [14]:
dataframes = []
for i, filename in enumerate(filenames): 
    print(i, filename.split("/")[-1])
    df = pd.read_csv(filename)
    dataframes.append(df)
    display(df)

0 data\assessments.csv


,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
...,...,...,...,...,...,...
201,GGG,2014J,37443,CMA,229.0,0.0
202,GGG,2014J,37435,TMA,61.0,0.0
203,GGG,2014J,37436,TMA,124.0,0.0
204,GGG,2014J,37437,TMA,173.0,0.0


1 data\courses.csv


,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240
5,BBB,2014B,234
6,CCC,2014J,269
7,CCC,2014B,241
8,DDD,2013J,261
9,DDD,2014J,262


2 data\studentAssessment.csv


...

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN
...,...,...,...,...,...
32588,GGG,2014J,2640965,-4.0,NaN
32589,GGG,2014J,2645731,-23.0,NaN
32590,GGG,2014J,2648187,-129.0,NaN
32591,GGG,2014J,2679821,-49.0,101.0


5 data\studentVle.csv


,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1
...,...,...,...,...,...,...
10655275,GGG,2014J,675811,896943,269,3
10655276,GGG,2014J,675578,896943,269,1
10655277,GGG,2014J,654064,896943,269,3
10655278,GGG,2014J,654064,896939,269,1


6 data\vle.csv


,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,NaN,NaN
1,546712,AAA,2013J,oucontent,NaN,NaN
2,546998,AAA,2013J,resource,NaN,NaN
3,546888,AAA,2013J,url,NaN,NaN
4,547035,AAA,2013J,resource,NaN,NaN
...,...,...,...,...,...,...
6359,897063,GGG,2014J,resource,NaN,NaN
6360,897109,GGG,2014J,resource,NaN,NaN
6361,896965,GGG,2014J,oucontent,NaN,NaN
6362,897060,GGG,2014J,resource,NaN,NaN


In [15]:
studentAssesment_df = dataframes[2]
studentInfo_df = dataframes[3]
courses_df = dataframes[1]
assesments_df = dataframes[0]
studentVle_df = dataframes[5]
vle_df = dataframes[6]
studentRegistration_df = dataframes[4]

In [16]:
studentInfo_df['result_encoded'] = studentInfo_df['final_result'].map({'Pass':1, 'Distinction':1, 'Withdrawn':0, 'Fail':0})

## Merge pandas frame to get data-format

In [17]:
# first join studentVLE with vle to get full view of vle activity
merged_vle_df = pd.merge(studentVle_df, vle_df, on = ['id_site', 'code_module', 'code_presentation'], how='left')

In [18]:
merged_vle_df

,code_module,code_presentation,id_student,id_site,date,sum_click,activity_type,week_from,week_to
0,AAA,2013J,28400,546652,-10,4,forumng,NaN,NaN
1,AAA,2013J,28400,546652,-10,1,forumng,NaN,NaN
2,AAA,2013J,28400,546652,-10,1,forumng,NaN,NaN
3,AAA,2013J,28400,546614,-10,11,homepage,NaN,NaN
4,AAA,2013J,28400,546714,-10,1,oucontent,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10655275,GGG,2014J,675811,896943,269,3,homepage,NaN,NaN
10655276,GGG,2014J,675578,896943,269,1,homepage,NaN,NaN
10655277,GGG,2014J,654064,896943,269,3,homepage,NaN,NaN
10655278,GGG,2014J,654064,896939,269,1,forumng,NaN,NaN


## All course data

In [19]:
# day and week boundaries
min_startday = -25
max_endday = 270
weeks = int((max_endday-min_startday)/7)

# resource activities
activities = list(set(list(merged_vle_df['activity_type'])))

In [20]:
len(activities)

20

In [69]:
def get_click_matrix(student_id, course_vle_df): 
    oneStudentVle = course_vle_df.loc[student_id, :]
    #click_matrix = np.zeros((len(activities), weeks))
    click_vector = np.zeros(len(activities))

    # insert clicks
    for i, vle_row in oneStudentVle.iterrows(): 
        # extract matrix index
        resource_index = activities.index(vle_row['activity_type'])
        #week_index = int((vle_row['date']-min_startday)/7)
        # add clicks to right cell
        #click_matrix[resource_index, week_index] += vle_row['sum_click']
        click_vector[resource_index] += vle_row['sum_click']

    return click_vector

def get_pass_fail(student_id, course_studentInfo_df): 
    result_encoded = int(course_studentInfo_df.loc[course_studentInfo_df['id_student']==student_id, 'result_encoded'])

    return result_encoded

In [70]:
# format the data
x, y = [], []
# iterate through all courses
for index, row in courses_df.iterrows(): 
    code_module = row['code_module']
    code_presentation = row['code_presentation']
    print(code_module, code_presentation)
    
    # extract only vle stats from the given course
    course_vle_df = merged_vle_df[(merged_vle_df.code_module==code_module)&(merged_vle_df.code_presentation==code_presentation)]

    # set index to id_student for easier extraction based on indivisual students later
    course_vle_df = course_vle_df.set_index('id_student')

    # extract student information for course module
    course_studentInfo_df = studentInfo_df.loc[(studentInfo_df.code_module==code_module)&(studentInfo_df.code_presentation==code_presentation), :]
    
    # format x and y data for model training 
    student_ids = list(set(course_vle_df.index))
    
    for student_id in student_ids: 
        try: 
            click_matrix = get_click_matrix(student_id, course_vle_df)
            result_encoded = get_pass_fail(student_id, course_studentInfo_df)
            x.append(click_matrix)
            y.append(result_encoded)
        except Exception as e: 
            pass
            #print(student_id)
            #print(e)
# numpy format
X = np.array(x)
y = np.array(y)

AAA 2013J
AAA 2014J
BBB 2013J
BBB 2014J
BBB 2013B
BBB 2014B
CCC 2014J
CCC 2014B
DDD 2013J
DDD 2014J
DDD 2013B
DDD 2014B
EEE 2013J
EEE 2014J
EEE 2014B
FFF 2013J
FFF 2014J
FFF 2013B
FFF 2014B
GGG 2013J
GGG 2014J
GGG 2014B


In [71]:
# reshape X to be one dimensional input datapoints
#X = np.reshape(X, (-1, X.shape[1]*X.shape[2]))

# split into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 3)

In [72]:
# print the counted y_train and y test
cnt_binary = Counter(list(y_train))
print("Training data: ", cnt_binary)
cnt_binary = Counter(list(y_test))
print("testing data: ", cnt_binary)

Training data:  Counter({1: 12340, 0: 10854})
testing data:  Counter({1: 3042, 0: 2757})


In [89]:
# fit model
clf = LogisticRegression(random_state=0, max_iter=600).fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.83      0.78      2757
           1       0.83      0.74      0.78      3042

    accuracy                           0.78      5799
   macro avg       0.78      0.78      0.78      5799
weighted avg       0.79      0.78      0.78      5799



C:\Users\perni\miniconda3\envs\tf_env\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [90]:
classification_report_all = classification_report(y_test, y_pred)

In [91]:
def print_latex_report(classification_report): 
    for line in classification_report.split("\n"): 
        if (len(line.split()) > 0): 
            for print_item in line.split()[:-1]: 
                print(print_item, " & ", end="")
            print(line.split()[-1], " \\\\")


In [92]:
print_latex_report(classification_report_all)

precision  & recall  & f1-score  & support  \\
0  & 0.74  & 0.83  & 0.78  & 2757  \\
1  & 0.83  & 0.74  & 0.78  & 3042  \\
accuracy  & 0.78  & 5799  \\
macro  & avg  & 0.78  & 0.78  & 0.78  & 5799  \\
weighted  & avg  & 0.79  & 0.78  & 0.78  & 5799  \\


## Half of the VLE data

In [77]:
duration_days = max_endday-min_startday
half_duration = int(duration_days/2)

In [78]:
def get_click_vector_half(student_id, course_vle_df): 
    oneStudentVle = course_vle_df.loc[student_id, :]
    #click_matrix = np.zeros((len(activities), weeks))
    click_vector = np.zeros(len(activities))

    # insert clicks
    for i, vle_row in oneStudentVle.iterrows(): 
        # do not include last half of click-activity
        day = int(vle_row['date'])
        if (day > half_duration): 
            continue
        
        # extract matrix index
        resource_index = activities.index(vle_row['activity_type'])
        # add clicks to right cell
        click_vector[resource_index] += vle_row['sum_click']

    return click_vector

In [79]:
# format the data
x_half, y = [], []
# iterate through all courses
for index, row in courses_df.iterrows(): 
    code_module = row['code_module']
    code_presentation = row['code_presentation']
    print(code_module, code_presentation)
    
    # extract only vle stats from the given course
    course_vle_df = merged_vle_df[(merged_vle_df.code_module==code_module)&(merged_vle_df.code_presentation==code_presentation)]

    # set index to id_student for easier extraction based on indivisual students later
    course_vle_df = course_vle_df.set_index('id_student')

    # extract student information for course module
    course_studentInfo_df = studentInfo_df.loc[(studentInfo_df.code_module==code_module)&(studentInfo_df.code_presentation==code_presentation), :]
    
    # format x and y data for model training 
    student_ids = list(set(course_vle_df.index))
    
    for student_id in student_ids: 
        try: 
            click_matrix = get_click_vector_half(student_id, course_vle_df)
            result_encoded = get_pass_fail(student_id, course_studentInfo_df)
            x_half.append(click_matrix)
            y.append(result_encoded)
        except Exception as e: 
            pass
            #print(student_id)
            #print(e)
# numpy format
X_half = np.array(x_half)
y = np.array(y)

AAA 2013J
AAA 2014J
BBB 2013J
BBB 2014J
BBB 2013B
BBB 2014B
CCC 2014J
CCC 2014B
DDD 2013J
DDD 2014J
DDD 2013B
DDD 2014B
EEE 2013J
EEE 2014J
EEE 2014B
FFF 2013J
FFF 2014J
FFF 2013B
FFF 2014B
GGG 2013J
GGG 2014J
GGG 2014B


In [80]:
# split into train and test set
X_half_train, X_half_test, y_train, y_test = train_test_split(X_half, y, test_size=0.2, random_state = 3)

In [81]:
# print the counted y_train and y test
cnt_binary = Counter(list(y_train))
print("Training data: ", cnt_binary)
cnt_binary = Counter(list(y_test))
print("testing data: ", cnt_binary)

Training data:  Counter({1: 12340, 0: 10854})
testing data:  Counter({1: 3042, 0: 2757})


In [86]:
# fit model
clf = LogisticRegression(random_state=0, max_iter=600).fit(X_half_train, y_train)

y_pred = clf.predict(X_half_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.68      0.78      0.72      2757
           1       0.77      0.66      0.71      3042

    accuracy                           0.72      5799
   macro avg       0.72      0.72      0.72      5799
weighted avg       0.73      0.72      0.72      5799



C:\Users\perni\miniconda3\envs\tf_env\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [87]:
classification_report_half = classification_report(y_test, y_pred)

In [88]:
print_latex_report(classification_report_half)

precision  & recall  & f1-score  & support  \\
0  & 0.68  & 0.78  & 0.72  & 2757  \\
1  & 0.77  & 0.66  & 0.71  & 3042  \\
accuracy  & 0.72  & 5799  \\
macro  & avg  & 0.72  & 0.72  & 0.72  & 5799  \\
weighted  & avg  & 0.73  & 0.72  & 0.72  & 5799  \\
